In [6]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.login_module import *
from modules.download_files_module import *
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *
from modules.creating_subscores_cast import *
from modules.creating_subscores_math import *
from modules.creating_subscores_ela import *

import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules.sql_query_module import SQL_query
import urllib
import sqlalchemy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day_year = today_date.strftime("%m_%d_%y")
print(formatted_month_day_year)
formatted_month_day_year = '06_18_24' #hardcoded for last day of downloads. 

download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day_year}'
pd.set_option('display.max_columns', None)

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')

# ---------------------------------POST SELENIUM PROCESS, STACKING & SENDING FILES----------------------------------
#Assumes files are downloaded & unzipped for the day
sbac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\sbac_{formatted_month_day_year}'
elpac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\elpac_{formatted_month_day_year}'

sbac_stack = stack_files(sbac_pdrive_dir, 'CAASPP') #This is where files are raw and stacked before transformation
elpac_stack = stack_files(elpac_pdrive_dir, 'ELPAC') #Green Dot Schools are pulled out of master file

elpac_stack = filter_on_full_cds_code(elpac_stack, 'CALPADSSchoolCode')
sbac_stack = filter_on_full_cds_code(sbac_stack, 'CALPADSSchoolCode')


c = get_cast_subscores(sbac_stack, 'CAST')
rw = get_ela_subscores_read_write(sbac_stack, 'SBAC')
es = get_ela_subscores_essay(sbac_stack, 'SBAC')
m = get_math_subscores_concepts_communicating(sbac_stack, 'SBAC') 
df = pd.concat([c, rw, es, m], ignore_index=True)


#CHECKS
#Can do df.groupby('SSID').size().unique() for Student Level Unit Tests
# s = sbac_stack[['SSID', 'RecordType','SmarterCompClaim1Score', 'SmarterCompClaim1PL', 'SmarterCompClaim2Score','SmarterCompClaim2PL', 'WERDEVEEL', 'WERCOV', 'WERPOR','Domain1Level', 'Domain2Level', 'Domain3Level' ]]
# s.loc[s['SSID'] == 	5403504677]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
07_05_24


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\ELPAC_SBAC_results_selenium\modules\sql_query_module.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con = conn)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\ELPAC_SBAC_results_selenium\modules\sql_query_module.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con = conn)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\ELPAC_SBAC_results_selenium\modules\sql_query_module.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or

# Changes 

- Map is as Essay if it is a blank - done
- Pull in TestGradeLevel - done
- Put in ProficiencyLevelCode for everything - done for all

In [ ]:

def send_to_sql(frame, file_name):

    dtypes, table_cols = SQL_query.get_dtypes(frame, 'DataTeamSandbox', f'{file_name}_Subscores')
    
    #Update the master table with a full replace, after assessing todays incoming records by each table
    try:
        frame.to_sql(f'{file_name}_Subscores', schema='dbo', con = SQL_query.engine, if_exists = 'replace', index = False, dtype=dtypes)
        logging.info(f"Sent data - {len(frame)} records to master table {file_name}_Scores")
    except Exception as e:
        logging.info(f'Unable to send data to {file_name}_Scores due to \n {e}')


send_to_sql(df, 'SBAC')